In [1]:
# packages

%matplotlib inline

from math import*
import matplotlib.pyplot as plt
import numpy as np
from array import*
import scipy.integrate as integrate
from scipy.interpolate import interp1d
import scipy.optimize as optimization
from scipy.special import erf
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
from array import array
from IPython.display import HTML
from IPython.display import display
from datetime import datetime, timedelta
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mdates

# Radon-222 to Plumbum-210 and Polonium-210 contamination tracker

## Flora Pennartz, June 2021

This program can be used to track down the accumulated amount of Radon-222 daughters Polonium-210 and Plumbum-210 when flushing the XAMSL setup with Radon. 

While fluhsing with Radon, some of the Radon decays into Plumbum. After the flushing, the Radon will be pumped out, but the Plumbum will stay, decaying into Polonium. The decaying Polonium could create noise in the setup.

The program first caculates the amout of Plumbum formed during the flushing, by solving the below formula for Radon and using the time duration of the flushing.

$$N = N_{0}e^{-\lambda t}$$

The number of Plumbum atoms is simpy the start Radon value minus the end Radon value. Then, the script takes the outcome for Plumbum and uses it for the same calculation; this time for Plumbum decaying into Polonium. The relation between activity and number of atoms is  given by the formula below.

$$A = \lambda N = -\frac{dN}{dt} $$

In [2]:
# constants

lambda_Rn222 = 2.1e-6 # decay constant Rn222 in s-1
lambda_Pb210 = 9.9e-10 # decay constant Pb210 in s-1
lambda_Po210 = 25.8e-8 # decay constant Po210 in s-1

In [3]:
# functions

def Rn222_to_Pb210 (start_time_flush, end_time_flush, start_flush_N_Rn222):
    
    start_time_unix = start_time_flush.timestamp()
    end_time_unix = end_time_flush.timestamp()  
    len_time_unix = end_time_unix - start_time_unix
    end_flush_N_Rn222 = int(start_flush_N_Rn222*np.exp(-lambda_Rn222*len_time_unix))
    end_flush_N_Pb210 = int(start_flush_N_Rn222 - end_flush_N_Rn222)
    
    return end_flush_N_Rn222, end_flush_N_Pb210

def Pb210_to_Po210 (start_decay_time, end_decay_time, end_flush_N_Pb210):
    
    start_time_unix = start_decay_time.timestamp()
    end_time_unix = end_decay_time.timestamp()  
    len_time_unix = end_time_unix - start_time_unix
    end_decay_N_Pb210 = int(end_flush_N_Pb210*np.exp(-lambda_Pb210*len_time_unix))
    end_decay_N_Po210 = int(end_flush_N_Pb210 - end_decay_N_Pb210)
   
    return end_decay_N_Pb210, end_decay_N_Po210

# What day do you want to check?
Say, you want to know the accumulated Polonium after so many Radon flushings on the 1st of January, 2025, midnight.

In [14]:
# format is (yyyy, [m]m, [d]d, [h]h, [m]m, [s]s)
end_decay_time = datetime(2025, 1, 1, 0, 0, 0)

# Radon flush 1
This function determines the amount of Radon and Plumbum after a certain flushing time: you only need to give the flush start time, flush end time and the number of Radon atoms at the start of the flush. I filled in an example.

In [6]:
start_time_flush_1 = datetime(2021, 6, 1, 9, 0, 0) # format dates is (yyyy, m, d, h, m, s))
end_time_flush_1 = datetime(2021, 6, 30, 17, 30, 30)
start_flush_N_Rn222_1 = 1000

end_flush_N_Rn222_1, end_flush_N_Pb210_1 = Rn222_to_Pb210 (start_time_flush_1, end_time_flush_1, start_flush_N_Rn222_1) 

print('With a start N value for Rn222:', start_flush_N_Rn222_1)
print('Number of Radon-222 atoms due to 1st flushing between', start_time_flush_1, 'and', end_time_flush_1,'will be:', end_flush_N_Rn222_1)
print('Number of Plumbum-210 atoms due to 1st flushing between', start_time_flush_1, 'and', end_time_flush_1,'will be:', end_flush_N_Pb210_1)

start_decay_time_1 = end_time_flush_1

end_decay_N_Pb210_1, end_decay_N_Po210_1 = Pb210_to_Po210 (start_decay_time_1, end_decay_time, end_flush_N_Pb210_1) 
print('Number of Plumbum-210 atoms due to 1st flush, left alone between', start_decay_time_1, 'and', end_decay_time,'will be:', end_decay_N_Pb210_1)
print('Number of Polonium-210 atoms due to 1st flush, left alone between', start_decay_time_1, 'and', end_decay_time,'will be:', end_decay_N_Po210_1)


With a start N value for Rn222: 1000
Number of Radon-222 atoms due to 1st flushing between 2021-06-01 09:00:00 and 2021-06-30 17:30:30 will be: 4
Number of Plumbum-210 atoms due to 1st flushing between 2021-06-01 09:00:00 and 2021-06-30 17:30:30 will be: 996
Number of Plumbum-210 atoms due to 1st flush, left alone between 2021-06-30 17:30:30 and 2025-01-01 00:00:00 will be: 892
Number of Polonium-210 atoms due to 1st flush, left alone between 2021-06-30 17:30:30 and 2025-01-01 00:00:00 will be: 104


# Radon flush 2
Here, you can add a second flushing. Example dates and times have been given.

In [7]:
# second Radon flushig
start_time_flush_2 = datetime(2022, 7, 1, 9, 0, 0)
end_time_flush_2 = datetime(2022, 7, 30, 17, 30, 30)
start_flush_N_Rn222_2 = 2000

end_flush_N_Rn222_2, end_flush_N_Pb210_2 = Rn222_to_Pb210 (start_time_flush_2, end_time_flush_2, start_flush_N_Rn222_2) 

print('With a start N value for Rn222:', start_flush_N_Rn222_2)
print('Number of Radon-222 atoms due to 2nd flushing between', start_time_flush_2, 'and', end_time_flush_2,'will be:', end_flush_N_Rn222_2)
print('Number of Plumbum-210 atoms due to 2nd flushing between', start_time_flush_2, 'and', end_time_flush_2,'will be:', end_flush_N_Pb210_2)

start_decay_time_2 = end_time_flush_2

end_decay_N_Pb210_2, end_decay_N_Po210_2 = Pb210_to_Po210 (start_decay_time_2, end_decay_time, end_flush_N_Pb210_2) 
print('Number of Plumbum-210 atoms due to 2nd flush, left alone between', start_decay_time_2, 'and', end_decay_time,'will be:', end_decay_N_Pb210_2)
print('Number of Polonium-210 atoms due to 2nd flush, left alone between', start_decay_time_2, 'and', end_decay_time,'will be:', end_decay_N_Po210_2)

With a start N value for Rn222: 2000
Number of Radon-222 atoms due to 2nd flushing between 2022-07-01 09:00:00 and 2022-07-30 17:30:30 will be: 9
Number of Plumbum-210 atoms due to 2nd flushing between 2022-07-01 09:00:00 and 2022-07-30 17:30:30 will be: 1991
Number of Plumbum-210 atoms due to 2nd flush, left alone between 2022-07-30 17:30:30 and 2025-01-01 00:00:00 will be: 1845
Number of Polonium-210 atoms due to 2nd flush, left alone between 2022-07-30 17:30:30 and 2025-01-01 00:00:00 will be: 146


# Radon flush 3
And a third!

In [8]:
# third Radon flushig

start_time_flush_3 = datetime(2023, 8, 14, 9, 0, 0)
end_time_flush_3 = datetime(2023, 8, 14, 17, 0, 0)
start_flush_N_Rn222_3 = 500

end_flush_N_Rn222_3, end_flush_N_Pb210_3 = Rn222_to_Pb210 (start_time_flush_3, end_time_flush_3, start_flush_N_Rn222_3) 

print('With a third flush start N value for Rn222:', start_flush_N_Rn222_3)
print('Number of Radon-222 atoms due to 3rd flushing between', start_time_flush_3, 'and', end_time_flush_3,'will be:', end_flush_N_Rn222_3)
print('Number of Plumbum-210 atoms due to 3rd flushing between', start_time_flush_3, 'and', end_time_flush_3,'will be:', end_flush_N_Pb210_3)

start_decay_time_3 = end_time_flush_3

end_decay_N_Pb210_3, end_decay_N_Po210_3 = Pb210_to_Po210 (start_decay_time_3, end_decay_time, end_flush_N_Pb210_3) 
print('Number of Plumbum-210 atoms due to 3rd flush, left alone between', start_decay_time_3, 'and', end_decay_time,'will be:', end_decay_N_Pb210_3)
print('Number of Polonium-210 atoms due to 3rd flush, left alone between', start_decay_time_3, 'and', end_decay_time,'will be:', end_decay_N_Po210_3)

With a third flush start N value for Rn222: 500
Number of Radon-222 atoms due to 3rd flushing between 2023-08-14 09:00:00 and 2023-08-14 17:00:00 will be: 470
Number of Plumbum-210 atoms due to 3rd flushing between 2023-08-14 09:00:00 and 2023-08-14 17:00:00 will be: 30
Number of Plumbum-210 atoms due to 3rd flush, left alone between 2023-08-14 17:00:00 and 2025-01-01 00:00:00 will be: 28
Number of Polonium-210 atoms due to 3rd flush, left alone between 2023-08-14 17:00:00 and 2025-01-01 00:00:00 will be: 2


# Radon flush 4
You can add another flushing by copying the "Radon flush 3" cell, fill in start_time_flush_3, end_time_flush_3, start_flush_N_Rn222_3, and change all 3's into 4's. Then, add end_decay_N_Pb210_4 and end_decay_N_Po210_4 in the accumulation cells. 

# Accumulation
Here you'll find the accumulation N and A for both Plumbum and Polonium, after all the flushes.

In [12]:
accum_N_Pb210 = end_decay_N_Pb210_1 + end_decay_N_Pb210_2 + end_decay_N_Pb210_3

print('On', end_decay_time, 'the accumulated number of Pb-210 atoms from all flushes since', start_time_flush_1,'will be', accum_N_Pb210)
print('The accompanying activity of Pb-210 will be', lambda_Pb210*accum_N_Pb210, 'Bq')

On 2025-01-01 00:00:00 the accumulated number of Pb-210 atoms from all flushes since 2021-06-01 09:00:00 will be 2765
The accompanying activity of Pb-210 will be 2.73735e-06 Bq


In [13]:
accum_N_Po210 = end_decay_N_Po210_1 + end_decay_N_Po210_2 + end_decay_N_Po210_3

print('On', end_decay_time, 'the accumulated number of Po-210 atoms from all flushes since', start_time_flush_1,'will be', accum_N_Po210)
print('The accompanying activity of Po-210 will be', lambda_Po210*accum_N_Po210, 'Bq')

On 2025-01-01 00:00:00 the accumulated number of Po-210 atoms from all flushes since 2021-06-01 09:00:00 will be 252
The accompanying activity of Po-210 will be 6.5016e-05 Bq
